# Udacity A/B Testing by Google

The final project of Udacity A/B Testing by Google.

[This project](https://classroom.udacity.com/courses/ud257/lessons/4126079196/concepts/42072285530923) is from Udacity free course [A/B Testing by Google](https://www.udacity.com/course/ab-testing--ud257).

---

Solution url: https://github.com/ZacksAmber/Udacity-A-B-Testing-by-Google

In [2]:
import pandas as pd
import numpy as np
import math
import scipy.stats as stats
from IPython.display import display, Latex

---

## Choosing Invariant Metrics

- Number of cookies
- Number of clicks
- Click-through-probability

---

## Calculating Standard Error

$\sigma^2 = p(1 - p)$

$SE = \sqrt{\frac{\sigma^2}{n}} = \frac{\sigma}{\sqrt{n}}$

---

|Metric Name|Metric Formula|$d_{min}$|Notation|Python Notation|Reason|
|:-:|:-:|:-:|:-:|:-:|:-:|
|Gross conversion|$\frac{C_{enrollment}}{C_{clicks}}$|0.01|Gross conversion|`gross_conversion`|The performance of a model|
|Retention|$\frac{C_{paid}}{C_{enrollment}}$|0.01|Retention|`retention`|The performance of a model|
|Net conversion|$\frac{C_{paid}}{C_{clicks}}$|0.0075|Net conversion|`net_conversion`|The performance of a model|

In [1]:
baseline = pd.read_csv('Final Project Baseline Values - Sheet1.csv')

baseline

NameError: name 'pd' is not defined

In [2]:
# Define baseline metrics
cookies = 40000
clicks = 3200
# enrollment is counted by user-id
enrollment = 660
# Enrollment probability
p_enrollment = enrollment / cookies
# Click-through-probability
CTP = clicks / cookies
gross_conversion = enrollment / clicks
retention = 0.53
net_conversion = gross_conversion * retention

# Dictonary of Gross conversion
GC = {}
GC['p'] = gross_conversion
GC['d_min'] = 0.01
GC['n'] = clicks

# Dictonary of Retention
R = {}
R['p'] = retention
R['d_min'] = 0.01
R['n'] = enrollment

# Dictonary of Net Conversion
NC = {}
NC['p'] = net_conversion
NC['d_min'] = 0.0075
NC['n'] = clicks

In [3]:
# display(Latex(r"$\sigma^2 = p(1 - p) \Rightarrow \sigma = \sqrt{p(1 - p)}$"))

def get_se(p, n):
    SE = np.sqrt(p * (1-p) / n)
    return round(SE, 4)

GC['SE'] = get_se(GC['p'], GC['n'])
R['SE'] = get_se(R['p'], R['n'])
NC['SE'] = get_se(NC['p'], NC['n'])

print(GC)
print(R)
print(NC)

{'p': 0.20625, 'd_min': 0.01, 'n': 3200, 'SE': 0.0072}
{'p': 0.53, 'd_min': 0.01, 'n': 660, 'SE': 0.0194}
{'p': 0.10931249999999999, 'd_min': 0.0075, 'n': 3200, 'SE': 0.0055}


## Sizing

Q: Choosing Number of Samples given Power

Using the analytic estimates of variance, how many pageviews total (across both groups) would you need to collect to adequately power the experiment? Use an alpha of 0.05 and a beta of 0.2. Make sure you have enough power for each metric.

---

### Familywise Error Rate (FWER)

**Less conservative multiple comparison methods**

The [Bonferroni correction](http://en.wikipedia.org/wiki/Bonferroni_correction) is a very simple method, but there are many other methods, including the [closed testing procedure](http://en.wikipedia.org/wiki/Closed_testing_procedure), the [Boole-Bonferroni bound](http://en.wikipedia.org/wiki/Bonferroni_bound), and the [Holm-Bonferroni method](http://en.wikipedia.org/wiki/Holm%E2%80%93Bonferroni_method). [This article](http://en.wikipedia.org/wiki/Multiple_comparisons_problem) on multiple comparisons contains more information, and this article contains more information about the false discovery rate (FDR), and methods for controlling that instead of the familywise error rate (FWER).

**Tracking multiple metrics**

Problem: Probability of any false positive increases as you increase number of metrics

Solution: Use higher confidence level for each metric

Method 1: Assume independence
- $\alpha_{overall} = 1 - (1 - \alpha_{individual})^n$

Method 2: Bonferroni correction
- simple
- no assumptions
- too conservative - guaranteed to give $\alpha_{overall}$ at least as small as specified
- $\alpha_{individual} = \frac{\alpha_{overall}}{n}$

for example:

$\alpha_{overall} = 0.05, n=3 \Rightarrow \alpha_{individual} = 0.0167$

In [4]:
# Z-Score

# Signifiance Level
alpha = 0.05
# n is the number of eveluation metrics
n = 3

# individual a of method 1
z1 = stats.norm.ppf(1 - (alpha**(1/n) / 2))
# z1 = stats.norm.ppf(1 - alpha / 2)

# individual a of method 2, Bonferroni correction
z2 = stats.norm.ppf(1 - (alpha/n) / 2)

print(z1)
print(z2)

0.8994685321132673
2.3939797998185104


In [5]:
# Margin of Error

# Set NumPy printionoptions
np.set_printoptions(suppress=True)
# Calculate margin of error
m1 = z1 * np.array([GC['SE'], R['SE'], NC['SE']])
m2 = z2 * np.array([GC['SE'], R['SE'], NC['SE']])

print(np.around(m1, 4))
print(np.around(m2, 4))

[0.0065 0.0174 0.0049]
[0.0172 0.0464 0.0132]


In [6]:
d_min = np.array([0.01, 0.01, 0.0075])

print(m1 < d_min)
print(m2 < d_min)

[ True False  True]
[False False False]


With respect to FWER control, the Bonferroni correction can be conservative if there are a large number of tests and/or the test statistics are positively correlated.

Thus, I decided not to use Bonferroni correction.

---

### Margin of Error

$\displaystyle MOE_{\gamma} = Z_{\gamma} \times SE = d_{min}$

---

### Sample Size per Variation

$\displaystyle  \hat p_{pool}= \frac{X_{cont}+X_{exp}}{N_{cont}+ N_{exp}}$

$\displaystyle SE_{pool} = \sqrt{\hat p_{pool} \times (1-\hat p_{pool}) \times (\frac{1}{N_{cont}}+\frac{1}{N_{exp}})}$

For calculating the minimize sample size of control and experiment, $N_{cont} = N_{exp}= N$, then we have:
$\displaystyle \hat p_{pool} = \frac{X_{cont}+X_{exp}}{N + N} = \frac{X_{cont}}{2N} + \frac{X_{exp}}{2N} = \frac{p_{cont}}{2} + \frac{p_{exp}}{2} = \frac{p_{cont} + p_{exp}}{2}$

Therefore:

$\displaystyle SE_{pool} = \sqrt{\hat p_{pool} \times (1-\hat p_{pool}) \times (\frac{1}{N_{cont}}+\frac{1}{N_{exp}})} = \sqrt{\hat p_{pool} \times (1-\hat p_{pool}) \times (\frac{1}{N} + \frac{1}{N})} = \sqrt{\hat p_{pool} \times (1-\hat p_{pool}) \times \frac{2}{N}}$

In this case, $d_{min}$ is equal to Margin of Error.

$d_{min} = p_{exp} - p_{cont}$

$\displaystyle MOE_{\gamma} = Z_{\gamma} \times SE = d_{min}$

$\displaystyle MOE_{\gamma} = Z_{\gamma} \cdot SE_{pool} \Rightarrow d_{min} = Z_{\gamma} \cdot \sqrt{\hat p_{pool} \times (1-\hat p_{pool}) \times \frac{2}{N}} \Rightarrow d_{min}^2 = Z_{\gamma} \cdot \hat p_{pool} \times (1-\hat p_{pool}) \times \frac{2}{N} \Rightarrow n = \frac{Z_{\gamma}^2 \cdot \hat p_{pool} \cdot (1 - \hat p_{pool}) \cdot 2}{d^2}$

$\displaystyle N = (z_{1-\alpha/2} + z_{1-\beta})^2 \left( \frac{\sigma}{\delta} \right)^2 = \frac{Z_{\gamma}^2 \cdot \hat p_{pool} \cdot (1 - \hat p_{pool}) \cdot 2}{d^2} \Rightarrow \mbox{for two-tailed test}$

$\displaystyle N = (z_{1-\alpha} + z_{1-\beta})^2 \left( \frac{\sigma}{\delta} \right)^2 = \frac{Z_{\gamma}^2 \cdot \hat p_{pool} \cdot (1 - \hat p_{pool}) \cdot 2}{d^2} \Rightarrow \mbox{for one-tailed test}$

---

### Calculating Number of Pageviews

The above formula is used by R power.prop.test and Evan's Awesome A/B Tools.

You can also visit [Evan's Awesome A/B Tools](https://www.evanmiller.org/ab-testing/sample-size.html) to calculate the power. 

However, there is a error between Evan's Awesome A/B Tools and the formula. Using the proper values as you need.

In [7]:
def get_z_score(sig):
    """
    Return z-score
    """
    return stats.norm.ppf(sig)

def sample_size(p1=0.10, p2=0.102, alpha=0.05, beta=0.2, alternative="two-sided", formula=False):
    """
    Return sample size per variation

    p1: float
        probability in control group
        Baseline conversion rate: bcr = p1
            An estimate of the metric being analyzed before making any changes
    p2: float
        probability in treatment group
        Minimum Detectable Effect: mde = p2 - p1 = d
            The Minimum Detectable Effect is the smallest effect that will be detected (1-β)% of the time.
    alpha: float
        The risk of rejecting a true hypothesis.
    beta: float
        The risk of accepting a false null hypothesis when a particular value of the alternative hypothesis is true.
        Power: 1 - β, also called sensitivity
        Sensitivity — the probability that the null hypothesis is not rejected when it should be

    Reference:
        https://itl.nist.gov/div898/handbook/prc/section2/prc222.htm
        https://jeffshow.com/caculate-abtest-required-sample-size.html
        https://www.evanmiller.org/ab-testing/sample-size.html
    """
    if alternative == "two-sided":
        z1 = get_z_score(1 - alpha/2)
        if formula is True:
            display(Latex(r"$H_0: P_A - P_B = 0$"))
            display(Latex(r"$H_1: P_A - P_B = d$"))
            display(Latex(r"$N = (z_{1-\alpha/2} + z_{1-\beta})^2 \left( \frac{\sigma}{\delta} \right)^2$"))
            # display(Latex(r"$n=\frac{(Z_{1-\alpha/2}\sqrt{2p_1 (1-p_1)}+Z_{1-\beta}\sqrt{p_1(1-p_1)+p_2(1-p_2)})^2}{d^2}$"))
    elif alternative == "one-sided":
        z1 = get_z_score(1 - alpha)
        if formula is True:
            display(Latex(r"$H_0: P_A - P_B = 0$"))
            display(Latex(r"$H_1: P_A - P_B = d$"))
            display(Latex(r"$N = (z_{1-\alpha} + z_{1-\beta})^2 \left( \frac{\sigma}{\delta} \right)^2$"))
            # display(Latex(r"$n=\frac{(Z_{1-\alpha}\sqrt{2p_1 (1-p_1)}+Z_{1-\beta}\sqrt{p_1(1-p_1)+p_2(1-p_2)})^2}{d^2}$"))
    else:
        raise ValueError("alternative must be one of ['two-sided', 'one-sided']")
    z2 = get_z_score(1 - beta)
    d = p2 - p1
    z = z1 + z2
    p_pool = (p1 + p2) / 2
    variance = p_pool * (1 - p_pool)
    n = z**2 * variance * 2 / d**2

    return int(np.round(n, 0))

Here is the data of three evaluation metrics.

|Metric Name|Metric Formula|$d_{min}$|Notation|Python Notation|Reason|
|:-:|:-:|:-:|:-:|:-:|:-:|
|Gross conversion|$\frac{C_{enrollment}}{C_{clicks}}$|0.01|Gross conversion|`gross_conversion`|The performance of a model|
|Retention|$\frac{C_{paid}}{C_{enrollment}}$|0.01|Retention|`retention`|The performance of a model|
|Net conversion|$\frac{C_{paid}}{C_{clicks}}$|0.0075|Net conversion|`net_conversion`|The performance of a model|

In [8]:
# Gross Conversion
print(GC)

# Retention
print(R)

# Net Conversion
print(NC)

{'p': 0.20625, 'd_min': 0.01, 'n': 3200, 'SE': 0.0072}
{'p': 0.53, 'd_min': 0.01, 'n': 660, 'SE': 0.0194}
{'p': 0.10931249999999999, 'd_min': 0.0075, 'n': 3200, 'SE': 0.0055}


In [9]:
# Sample Size from function sample_size()
GC['Sample Size'] = sample_size(GC['p'], GC['p']+GC['d_min'])
R['Sample Size'] = sample_size(R['p'], R['p']+R['d_min'])
NC['Sample Size'] = sample_size(NC['p'], NC['p']+NC['d_min'])

print(GC)
print(R)
print(NC)

{'p': 0.20625, 'd_min': 0.01, 'n': 3200, 'SE': 0.0072, 'Sample Size': 26156}
{'p': 0.53, 'd_min': 0.01, 'n': 660, 'SE': 0.0194, 'Sample Size': 39052}
{'p': 0.10931249999999999, 'd_min': 0.0075, 'n': 3200, 'SE': 0.0055, 'Sample Size': 27985}


In [10]:
# Sample Size from Evan's Awesome A/B Tools
GC['Sample Size'] = 25835
R['Sample Size'] = 39115
NC['Sample Size'] = 27413

print(GC)
print(R)
print(NC)

{'p': 0.20625, 'd_min': 0.01, 'n': 3200, 'SE': 0.0072, 'Sample Size': 25835}
{'p': 0.53, 'd_min': 0.01, 'n': 660, 'SE': 0.0194, 'Sample Size': 39115}
{'p': 0.10931249999999999, 'd_min': 0.0075, 'n': 3200, 'SE': 0.0055, 'Sample Size': 27413}


Before calcuating the minimum pageviews(unique cookies) for each metric, we need to know that the above minimum sample size is `clicks` or `enrollment`(counted by user-id). Therefore, we need to divide them by probability.
- CTP = clicks / cookies
- p_enrollment = enrollment / cookies

For `Gross conversion`, the formula is $\frac{C_{enrollment}}{C_{clicks}}$. The unit of diversion is `clicks`. So the number of minimum pageviews is $\displaystyle clicks \div CTP \cdot 2 \Rightarrow clicks \div \frac{clicks}{cookies} \cdot 2$

In [19]:
GC['pageviews'] = int(round(GC['Sample Size'] / CTP * 2, 0))
GC['pageviews']

645875

For `Retention`, the formula is $\frac{C_{paid}}{C_{enrollment}}$. The unit of diversion is `enrollment`(user-id). So the number of minimum pageviews is $\displaystyle enrollment \div p\_enrollment \cdot 2 \Rightarrow enrollment \div \frac{enrollment}{cookies} \cdot 2$

In [20]:
R['pageviews'] = int(round(R['Sample Size'] / p_enrollment * 2))
R['pageviews'] 

4741212

For `Net conversion`, the formula is $\frac{C_{paid}}{C_{clicks}}$. The unit of diversion is `clicks`. So the number of minimum pageviews is $\displaystyle clicks \div CTP \cdot 2 \Rightarrow clicks \div \frac{clicks}{cookies} \cdot 2$

In [21]:
NC['pageviews'] = int(round(NC['Sample Size'] / CTP * 2))
NC['pageviews']

685325

In [22]:
max([GC['pageviews'], R['pageviews'], NC['pageviews']])

4741212

The maximum pageviews for two gruops from `Gross conversion`, `Retention`, and `Net conversion` is 4741212.

---

### Choosing Duration and Exposure

Q: What percentage of Udacity's traffic would you divert to this experiment (assuming there were no other experiments you wanted to run simultaneously)? Is the change risky enough that you wouldn't want to run on all traffic?

Q: Given the percentage you chose, how long would the experiment take to run, using the analytic estimates of variance? If the answer is longer than a few weeks, then this is unreasonably long, and you should reconsider an earlier decision.

- **Udacity currently have 40,000 pageviews (unique cookies) to view course overview page per day.**
- For calculating the minimum duration, we divert 100% daily cookies for this experiment. You can also modify the fraction.

In [50]:
# Define cookies fraction
cookies_fraction = 1

In [51]:
# Duration of Gross conversion
GC['duration'] = math.ceil(GC['pageviews'] / (cookies * cookies_fraction))

# Duration of Retention
R['duration'] = math.ceil(R['pageviews'] / (cookies * cookies_fraction))

# Duration of Net conversion
NC['duration'] = math.ceil(NC['pageviews'] / (cookies * cookies_fraction))

In [55]:
duration_table = pd.DataFrame({
    'Metric Name': ['Gross conversion', 'Retention', 'Net conversion'],
    'Minimum clicks/enrollment': [GC['Sample Size'], R['Sample Size'], NC['Sample Size']], 
    'Minimum pageviews': [GC['pageviews'], R['pageviews'], NC['pageviews']],
    'Fraction of experiment traffic': [cookies_fraction] * 3,
    'Duration': [GC['duration'], R['duration'], NC['duration']]
})

duration_table

,Metric Name,Minimum clicks/enrollment,Minimum pageviews,Fraction of experiment traffic,Duration
0,Gross conversion,25835,645875,1,17
1,Retention,39115,4741212,1,119
2,Net conversion,27413,685325,1,18


A: According to above table, when we assign 100% daily traffic to this experiment, we have the minimum duration. We only have 40,000 pageviews (unique cookies) per day. Therefore, we need to abandon `Retention` as an qualified evaulation metric due to the too long-running experiment.

The minimum duration for `Gross conversion` and `Net conversion` are 17 days and 18 days. For ensuring the effectiveness of the experiment, we need run the experiment at least 18 days.


---

## Sanity Checks

For each metric that you choose as an invariant metric, compyte a 95% confidence interval for the value you expect to observe. Enter the upper and lower bounds, and the observed value, all to 4 decimal places.

Invariant Metrics:
- cookies(pageviews)
- clicks
- CTP(Clicks-through-probability, whichs is clicks / cookies)

The experiment data can be found in [this spreadsheet](https://docs.google.com/spreadsheets/d/1Mu5u9GrybDdska-ljPXyBjTpdZIUev_6i7t4LRDfXM8/edit#gid=0); use this information to answer the analysis questions. Note that control data and experiment data are on separate tabs of the spreadsheet.

If we have $p$, use $p$ instead of $\hat p$.

- $\displaystyle H_0: p = p_0$
- $\displaystyle H_1: p \neq p_0$
- $\displaystyle \sigma^2 = \hat p (1 - \hat p)$
- $\displaystyle \displaystyle SE = \sqrt{\frac{\sigma^2}{n}}$
- $\displaystyle Z_{\gamma} = Z_{1 - \alpha}$
- $\displaystyle Z_{\gamma} = Z_{1 - \frac{\alpha}{2}}$
- $\displaystyle MOE_{\gamma} = Z_{\gamma} \cdot SE$
- $\displaystyle CI = \hat p \pm MOE_{\gamma}$

In [67]:
# Import Dataset of control
control = pd.read_csv('Final Project Results - Control.csv')

control

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0
...,...,...,...,...,...
32,"Wed, Nov 12",10134,801,NaN,NaN
33,"Thu, Nov 13",9717,814,NaN,NaN
34,"Fri, Nov 14",9192,735,NaN,NaN
35,"Sat, Nov 15",8630,743,NaN,NaN


In [68]:
# Import Dataset of experiment
experiment = pd.read_csv('Final Project Results - Experiment.csv')

experiment

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0
...,...,...,...,...,...
32,"Wed, Nov 12",10042,802,NaN,NaN
33,"Thu, Nov 13",9721,829,NaN,NaN
34,"Fri, Nov 14",9304,770,NaN,NaN
35,"Sat, Nov 15",8668,724,NaN,NaN


In [331]:
# Summarize control, skip 0, which is Date
control_sum = list(control.iloc[:, 1:].sum())
# Summarize experiment, skip 0, which is Date
experiment_sum = list(experiment.iloc[:, 1:].sum())

# Define DataFrame santiy_checks
sanity_checks = pd.DataFrame({
    'A': control_sum,
    'B': experiment_sum,
}, index=['cookies', 'clicks', 'enrollments', 'payments'])

# Format the DataFrame
sanity_checks = sanity_checks.astype({'A': 'int32', 'B': 'int32'})

sanity_checks

,A,B
cookies,345543,344660
clicks,28378,28325
enrollments,3785,3423
payments,2033,1945


In [356]:
sanity_checks['Total'] = sanity_checks.A + sanity_checks.B
# For cookies and clicks, P = 0.5 for evenly spilting the traffic to control and experiment;
# For enrollments and payments, P = 0.5, the null hypothesis is that there is no difference between control and experiment.
sanity_checks['p'] = 0.5
# Variance
sanity_checks['Var'] = sanity_checks['p'] * (1 - sanity_checks['p'])
# Standard Error
sanity_checks['SE'] = np.sqrt(sanity_checks['Var'] / sanity_checks['Total'])
# Significance Level
sanity_checks['alpha'] = 0.05
# Margin of Error
sanity_checks['MOE'] = stats.norm.ppf(1 - sanity_checks['alpha']/2) * sanity_checks['SE']
# CI lower bound
sanity_checks['CI lower bound'] = sanity_checks['p'] - sanity_checks['MOE']
# CI upper bound
sanity_checks['CI upper bound'] = sanity_checks['p'] + sanity_checks['MOE']
# Observed
sanity_checks['Observed'] = sanity_checks['A'] / sanity_checks['Total']
# Pass or not
sanity_checks['Pass'] = sanity_checks.apply(lambda x: (x['CI lower bound'] <= x['Observed']) and (x['Observed'] <= x['CI upper bound']),axis=1)
# Difference
sanity_checks['d'] = abs((sanity_checks['B'] - sanity_checks['A']) / sanity_checks['Total'])

sanity_checks.round(4)

,A,B,Total,p,Var,SE,alpha,MOE,CI lower bound,CI upper bound,Observed,Pass,d
cookies,345543,344660,690203,0.5,0.25,0.0006,0.05,0.0012,0.4988,0.5012,0.5006,True,0.0013
clicks,28378,28325,56703,0.5,0.25,0.0021,0.05,0.0041,0.4959,0.5041,0.5005,True,0.0009
enrollments,3785,3423,7208,0.5,0.25,0.0059,0.05,0.0115,0.4885,0.5115,0.5251,False,0.0502
payments,2033,1945,3978,0.5,0.25,0.0079,0.05,0.0155,0.4845,0.5155,0.5111,True,0.0221


In [578]:
# Define function sanity_check to calculate margin of error, confidence interval
def one_sample_prop(N, X, alpha=0.05, p=0, alternative="two-sided", round=4, formula=False):
    """
    One-Sample Proportion for Sanity Check.

    Return
        - Margin of Error
        - Left edge of Confidence Interval
        - Right edge of Confidence Interval
        - Observed probability
        - Statistical Significance (If CI contains 0)
        - Practical Significance (If p_hat in CI)

    N: int
        Control data and Experiment data
    X: int
        Control data
    alpha: float
        Default: 0.05
        Significance Level: between 0 to 1
    p: float
        Population probability. Leave it blank if it is unknown.
    alternative: str
        Default: "two-sided"
        One of "two-sided" or "one-sided"
    round: int
        Default: 4
        Round a number to a given precision in decimal digits.
    formula: boolean
        Default: False
        Display formula.

    Reference:
        [Binomial proportion confidence interval](https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval)
        [Hypothesis Testing for One Sample Proportion](https://online.stat.psu.edu/stat800/lesson/5/5.2)

    >>> binomial_CI(N=1000, X=450, p=0.5, alternative="two-sided", round=4)
    """

    # Validate significance level - alpha
    if (0 <= alpha <= 1) is False:
        raise ValueError(f'significance level (alpha) must be in the range of [0, 1]')

    if alternative == "two-sided":
        z1 = get_z_score(1 - alpha/2)
        if formula is True:
            display(Latex(r"$\displaystyle H_0: p = p_0$"))
            display(Latex(r"$\displaystyle H_1: p \neq p_0$"))
            display(Latex(r"$\displaystyle \sigma^2 = \hat p (1 - \hat p)$"))
            display(Latex(r"$\displaystyle \displaystyle SE = \sqrt{\frac{\sigma^2}{n}}$"))
            display(Latex(r"$\displaystyle Z_{\gamma} = Z_{1 - \frac{\alpha}{2}}$"))
            display(Latex(r"$\displaystyle MOE_{\gamma} = Z_{\gamma} \cdot SE$"))
            display(Latex(r"$\displaystyle CI = \hat p \pm MOE_{\gamma}$"))
    elif alternative == "one-sided":
        z1 = get_z_score(1 - alpha)
        if formula is True:
            display(Latex(r"$\displaystyle H_0: p = p_0$"))
            display(Latex(r"$\displaystyle H_1: p \neq p_0$"))
            display(Latex(r"$\displaystyle \sigma^2 = \hat p (1 - \hat p)$"))
            display(Latex(r"$\displaystyle \displaystyle SE = \sqrt{\frac{\sigma^2}{n}}$"))
            display(Latex(r"$\displaystyle Z_{\gamma} = Z_{1 - \alpha}$"))
            display(Latex(r"$\displaystyle MOE_{\gamma} = Z_{\gamma} \cdot SE$"))
            display(Latex(r"$\displaystyle CI = \hat p \pm MOE_{\gamma}$"))
    else:
        raise ValueError("alternative must be one of ['two-sided', 'one-sided']")
    
    # Estimated probability of success, which is the sample statistic, the midpoint of Confidence Interval
    p_hat = X / N

    # If population probability is unknown, the program will use estimated probability for calculation.
    if p == 0:
        p = p_hat

    # When not proper for normal distribution assumption
    if N * p <= 5 or N * (1 - p) <= 5:
        return False

    # Standard Error of Two-tailed test
    SE = math.sqrt(p * (1 - p) / N)

    # Margin of Error
    m = z * SE

    # Left & Right boundary of Confidence Interval
    CI_left, CI_right = p - m, p + m

    # Statistical Significance
    if CI_left < 0 < CI_right:
        statistical_significance = False
    else:
        statistical_significance = True

    # Pratical Significance
    if CI_left <= p_hat <= CI_right:
        pratical_significance = False
    else:
        pratical_significance = True

    return np.round(m, round), np.round(CI_left, round), np.round(CI_right, round), np.round(p_hat, round), statistical_significance, pratical_significance

In [606]:
df_sanity_checks = pd.DataFrame(
    columns=['Margin of Error', 'CI lower bound', 'CI upper bound', 'Observed', 'Statistical Significance', 'Practical Significance'], 
    index=['cookies', 'clicks', 'CTP']
    )

In [607]:
# Hypothesis Testing for One-Sample Proportions (Sanity Check)
# cookies
# p is 0.5
result = one_sample_prop(N=sanity_checks.loc['cookies', 'Total'], X=sanity_checks.loc['cookies', 'A'], p=0.5, round=4, formula=False)

df_sanity_checks.loc['cookies', :] = result

In [608]:
# Hypothesis Testing for One-Sample Proportions (Sanity Check)
# clicks
# p is 0.5
result = one_sample_prop(N=sanity_checks.loc['clicks', 'Total'], X=sanity_checks.loc['clicks', 'A'], p=0.5, round=4)

df_sanity_checks.loc['clicks', :] = result

In [609]:
# Hypothesis Testing for One-Sample Proportions (Sanity Check)
# CTP
# p is unknown
result = one_sample_prop(N=sanity_checks.loc['cookies', 'A'], X=sanity_checks.loc['clicks', 'A'], round=4)

df_sanity_checks.loc['CTP', :] = result

In [610]:
df_sanity_checks

,Margin of Error,CI lower bound,CI upper bound,Observed,Statistical Significance,Practical Significance
cookies,0.0012,0.4988,0.5012,0.5006,True,False
clicks,0.0041,0.4959,0.5041,0.5005,True,False
CTP,0.0009,0.0812,0.083,0.0821,True,False


---

## Effect Size Tests

For each of your evaluation metrics, comput a confidence interval around the difference.

Evaluation Metrics:
- Gross conversion
- Net conversion

The experiment data can be found in [this spreadsheet](https://docs.google.com/spreadsheets/d/1Mu5u9GrybDdska-ljPXyBjTpdZIUev_6i7t4LRDfXM8/edit#gid=0); use this information to answer the analysis questions. Note that control data and experiment data are on separate tabs of the spreadsheet.

> [Hypothesis Testing for Two-Sample Proportions](https://online.stat.psu.edu/stat800/lesson/5/5.5)

- $\displaystyle H_0: P_A - P_B = 0$
- $\displaystyle H_1: P_A - P_B = d$
- $\displaystyle \hat d = p_2 - p_1$
- $\displaystyle \hat p = \frac{x_1 + x_2}{n_1 + n_2}$
- $\displaystyle \sigma^2 = \hat p (1 - \hat p)$
- $\displaystyle SE = \sqrt{\sigma^2 \cdot (\frac{1}{n_1} + \frac{1}{n_2})}$
- $\displaystyle Z_{\gamma} = Z_{1 - \frac{\alpha}{2}}$
- $\displaystyle Z_{\gamma} = Z_{1 - \alpha}$
- $\displaystyle MOE_{\gamma} = Z_{\gamma} \times SE$
- $\displaystyle CI = \hat d \pm MOE_{\gamma}$

|Metric Name|Metric Formula|$d_{min}$|Notation|Python Notation|Reason|
|:-:|:-:|:-:|:-:|:-:|:-:|
|Gross conversion|$\frac{C_{enrollment}}{C_{clicks}}$|0.01|Gross conversion|`gross_conversion`|The performance of a model|
|Retention|$\frac{C_{paid}}{C_{enrollment}}$|0.01|Retention|`retention`|The performance of a model|
|Net conversion|$\frac{C_{paid}}{C_{clicks}}$|0.0075|Net conversion|`net_conversion`|The performance of a model|

In [418]:
# Summarize control, skip 0, which is Date. And filter not null rows.
control_notnull_sum = list(control[control['Enrollments'].notnull() & control['Payments'].notnull()].iloc[:, 1:].sum())
# Summarize experiment, skip 0, which is Date. And filter not null rows.
experiment_notnull_sum = list(experiment[experiment['Enrollments'].notnull() & experiment['Payments'].notnull()].iloc[:, 1:].sum())

In [499]:
# Define DataFrame santiy_checks
analysis_test = pd.DataFrame({
    'A': control_notnull_sum,
    'B': experiment_notnull_sum,
}, index=['cookies', 'clicks', 'enrollments', 'payments'])

# Format the DataFrame
analysis_test = analysis_test.astype({'A': 'int32', 'B': 'int32'})

analysis_test

,A,B
cookies,212163,211362
clicks,17293,17260
enrollments,3785,3423
payments,2033,1945


In [500]:
analysis_test.loc['Gross conversion', :] = analysis_test.loc['enrollments', :] / analysis_test.loc['cookies', :]
analysis_test.loc['Retention', :] = analysis_test.loc['payments', :] / analysis_test.loc['enrollments', :]
analysis_test.loc['Net conversion', :] = analysis_test.loc['payments', :] / analysis_test.loc['cookies', :]
analysis_test['d_min'] = [3000, 240, None, None, 0.01, 0.01, 0.0075]
analysis_test['Total'] = analysis_test['A'] + analysis_test['B']

analysis_test.round(4)

,A,B,d_min,Total
cookies,212163.0000,211362.0000,3000.0000,423525.0000
clicks,17293.0000,17260.0000,240.0000,34553.0000
enrollments,3785.0000,3423.0000,NaN,7208.0000
payments,2033.0000,1945.0000,NaN,3978.0000
Gross conversion,0.0178,0.0162,0.0100,0.0340
Retention,0.5371,0.5682,0.0100,1.1053
Net conversion,0.0096,0.0092,0.0075,0.0188


In [593]:
def two_sample_prop(n1, n2, x1, x2, d_min, alpha=0.05, alternative="two-sided", round=4, formula=False):
    """
    Two-Sample Proportion for analysis.

    Return
        - Margin of Error
        - Left edge of Confidence Interval
        - Right edge of Confidence Interval
        - Observed probability
        - Statistical Significance (If CI contains 0)
        - Practical Significance (If CI > d_min or CI < -d_min)

    n1: int
        Sample size of group A
    n2: int
        Sample size of group B
    x1: int
        Successful count of group A
    x2: int
        Successful count of group B
    d_min: float
        Minimum Effect Difference
    alpha: float
        Default: 0.05
        Significance Level: between 0 to 1
    alternative: str
        Default: "two-sided"
        One of "two-sided" or "one-sided"
    round: int
        Default: 4
        Round a number to a given precision in decimal digits.
    formula: boolean
        Default: False
        Display formula.

    Reference
        [Hypothesis Testing for Two-Sample Proportions](https://online.stat.psu.edu/stat800/lesson/5/5.5)

    >>> analysis_test()
    """

    if alternative == "two-sided":
        z1 = get_z_score(1 - alpha/2)
        if formula is True:
            display(Latex(r"$\displaystyle H_0: P_A - P_B = 0$"))
            display(Latex(r"$\displaystyle H_1: P_A - P_B = d$"))
            display(Latex(r"$\displaystyle \hat d = p_2 - p_1$"))
            display(Latex(r"$\displaystyle \hat p = \frac{x_1 + x_2}{n_1 + n_2}$"))
            display(Latex(r"$\displaystyle \sigma^2 = \hat p (1 - \hat p)$"))
            display(Latex(r"$\displaystyle SE = \sqrt{\sigma^2 \cdot (\frac{1}{n_1} + \frac{1}{n_2})}$"))
            display(Latex(r"$\displaystyle Z_{\gamma} = Z_{1 - \frac{\alpha}{2}}$"))
            display(Latex(r"$\displaystyle MOE_{\gamma} = Z_{\gamma} \times SE$"))
            display(Latex(r"$\displaystyle CI = \hat d \pm MOE_{\gamma}$"))
    elif alternative == "one-sided":
        z1 = get_z_score(1 - alpha)
        if formula is True:
            display(Latex(r"$\displaystyle H_0: P_A - P_B = 0$"))
            display(Latex(r"$\displaystyle H_1: P_A - P_B = d$"))
            display(Latex(r"$\displaystyle \hat d = p_2 - p_1$"))
            display(Latex(r"$\displaystyle \hat p = \frac{x_1 + x_2}{n_1 + n_2}$"))
            display(Latex(r"$\displaystyle \sigma^2 = \hat p (1 - \hat p)$"))
            display(Latex(r"$\displaystyle SE = \sqrt{\sigma^2 \cdot (\frac{1}{n_1} + \frac{1}{n_2})}$"))
            display(Latex(r"$\displaystyle Z_{\gamma} = Z_{1 - \alpha}$"))
            display(Latex(r"$\displaystyle MOE_{\gamma} = Z_{\gamma} \times SE$"))
            display(Latex(r"$\displaystyle CI = \hat d \pm MOE_{\gamma}$"))
    else:
        raise ValueError("alternative must be one of ['two-sided', 'one-sided']")

    # A is control; B is experiment
    p1 = x1 / n1
    p2 = x2 / n2
    p_hat = (x1 + x2) / (n1 + n2)
    SE_pool = np.sqrt(p_hat * (1 - p_hat) * (1/n1 + 1/n2))
    # Difference, the CI middle point
    d_hat = p2 - p1

    # Margin of Error
    m = z * SE_pool

    # Confidence Interval
    CI_left, CI_right = d_hat - m, d_hat + m

    # Statistical Significance
    if CI_left < 0 < CI_right:
        statistical_significance = False
    else:
        statistical_significance = True

    # Pratical Significance
    if CI_right < -d_min or CI_left > d_min:
        pratical_significance = True
    else:
        pratical_significance = False
    
    return np.round(m, round), np.round(CI_left, round), np.round(CI_right, round), np.round(d_hat, round), statistical_significance, pratical_significance

In [ ]:
df_analysis = pd.DataFrame(
    columns=['Margin of Error', 'CI lower bound', 'CI upper bound', 'Observed', 'Statistical Significance', 'Practical Significance'], 
    index=['Gross conversion', 'Net conversion']
    )

In [601]:
# Hypothesis Testing for Two-Sample Proportions
# Gross conversion
result = two_sample_prop(n1=analysis_test.loc['clicks', 'A'], n2=analysis_test.loc['clicks', 'B'], x1=analysis_test.loc['enrollments', 'A'], x2=analysis_test.loc['enrollments', 'B'], d_min=0.01, round=4, formula=False)
# -0.012 < -0.01 (d_min)

df_analysis.loc['Gross conversion', :] = result

In [602]:
# Hypothesis Testing for Two-Sample Proportions
# Net conversion
result = two_sample_prop(n1=analysis_test.loc['clicks', 'A'], n2=analysis_test.loc['clicks', 'B'], x1=analysis_test.loc['payments', 'A'], x2=analysis_test.loc['payments', 'B'], d_min=0.0075, round=4, formula=False)

df_analysis.loc['Net conversion', :] = result

In [603]:
df_analysis

,Margin of Error,CI lower bound,CI upper bound,Observed,Statistical Significance,Practical Significance
Gross conversion,0.0086,-0.0291,-0.012,-0.0206,True,True
Net conversion,0.0067,-0.0116,0.0019,-0.0049,False,False


---

## Sign Test

Run a  sign test on each of your evaluation metrics using the day-by-day data. Enter each pvalue, and indicate whether each result is statistically significant.

> [Binomial test](https://en.wikipedia.org/wiki/Binomial_test)

- $\displaystyle H_0: \pi = \pi_0$
- $\displaystyle Pr(X = k) = {n \choose k} p^k (1 - p)^{n-k}$
- one-tailed
    - $\displaystyle p = \sum^k_{i=0} Pr(X = i) = \sum^k_{i=0} {n \choose i} p^k (1 - p)^{n-i}$
- two-tailed
    - $\displaystyle p = \sum_{i \in \mathcal{I}} Pr(X = i) = \sum_{i \in \mathcal{I}} {n \choose i} p^k (1 - p)^{n-i}$



|Metric Name|Metric Formula|$d_{min}$|Notation|Python Notation|Reason|
|:-:|:-:|:-:|:-:|:-:|:-:|
|Gross conversion|$\frac{C_{enrollment}}{C_{clicks}}$|0.01|Gross conversion|`gross_conversion`|The performance of a model|
|Retention|$\frac{C_{paid}}{C_{enrollment}}$|0.01|Retention|`retention`|The performance of a model|
|Net conversion|$\frac{C_{paid}}{C_{clicks}}$|0.0075|Net conversion|`net_conversion`|The performance of a model|

In [674]:
# Define function binom_test returning binomial test p-value or binomial probability
def binom_test(x, n, p=0.5, alternative="two-sided", cumulative=True, round=4, formula=False):
    """
    Binomial test

    Return
        - When cumulative=False, return Probability.
        - When cumulative=True, return cumulative Probabiltiy, which is p-value.

    Return expected different d, margin of error m, CI left boundary CI_left, CI right boundary CI_right.

    x: int
        The number of successes, or if x has length 2, it is the number of successes and the number of failures.
    n: int
        The number of trials. This is ignored if x gives both the number of successes and failures.
    p: float, optional
        The hypothesized probability of success. 0 <= p <= 1. The default value is p = 0.5.
    alternative: {‘two-sided’, ‘greater’, ‘less’}, optional
        Indicates the alternative hypothesis. The default value is ‘two-sided’.
    cumulative: boolean, optional
        Return the cumulative probability. The default value is True.
    round: int, optional
        Round a number to a given precision in decimal digits. The default value is 4.
    formula: boolean
        Display formula. The default value is False.

    Reference
        [Binomial test](https://en.wikipedia.org/wiki/Binomial_test)
        [scipy.stats.binom_test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom_test.html#scipy.stats.binom_test)
        [Tests with Matched Samples](https://sphweb.bumc.bu.edu/otlt/mph-modules/bs/bs704_nonparametric/BS704_Nonparametric5.html)

    >>> binom_test(n=14, x=9, p=0.5, alternative='two-sided', cumulative=True, round=4, formula=True)
    >>> binom_test(n=14, x=9, p=0.5, alternative='greater', cumulative=True, round=4, formula=True)
    >>> binom_test(n=14, x=9, p=0.5, cumulative=False, round=4, formula=True)
    """

    if alternative == "two-sided":
        if formula is True:
            display(Latex(r"$\displaystyle H_0: \pi = \pi_0$"))
            display(Latex(r"$\displaystyle H_1: \pi \neq \pi_0$"))
            display(Latex(r"$\displaystyle Pr(X = k) = {n \choose k} p^k (1 - p)^{n-k}$"))
            display(Latex(r"$\displaystyle p = \sum_{i \in \mathcal{I}} Pr(X = i) = \sum_{i \in \mathcal{I}} {n \choose i} p^k (1 - p)^{n-i}$"))
    elif alternative == "greater":
        if formula is True:
            display(Latex(r"$\displaystyle H_0: \pi = \pi_0$"))
            display(Latex(r"$\displaystyle H_1: \pi \neq \pi_0$"))
            display(Latex(r"$\displaystyle Pr(X = k) = {n \choose k} p^k (1 - p)^{n-k}$"))
            display(Latex(r"$\displaystyle p = \sum^k_{i=0} Pr(X = i) = \sum^k_{i=0} {n \choose i} p^k (1 - p)^{n-i}$"))
    elif alternative == "less":
        if formula is True:
            display(Latex(r"$\displaystyle H_0: \pi = \pi_0$"))
            display(Latex(r"$\displaystyle H_1: \pi \neq \pi_0$"))
            display(Latex(r"$\displaystyle Pr(X = k) = {n \choose k} p^k (1 - p)^{n-k}$"))
            display(Latex(r"$\displaystyle p = \sum^k_{i=0} Pr(X = i) = \sum^k_{i=0} {n \choose i} p^k (1 - p)^{n-i}$"))
    else:
        raise ValueError("alternative must be one of ['two-sided', 'greater', 'less']")

    if cumulative is True:
        return np.round(stats.binom_test(x, n, p, alternative), round)
    else:
        return np.round(math.comb(n, k) * p**k * (1-p)**(n-k), round)

In [691]:
notnull_control = control[control.Payments.notnull()]
notnull_experiment = experiment[experiment.Payments.notnull()]

In [722]:
# The number of successful trails, when the Gross conversion in experiment is greater than it is in the control
x = sum(notnull_experiment.Enrollments / notnull_experiment.Clicks > notnull_control.Enrollments / notnull_control.Clicks)
n = len(notnull_experiment)

binom_test(x=x, n=n, p=0.5, cumulative=True, alternative='two-sided', formula=True)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

0.0026

In [723]:
# The number of successful trails, when the Net conversion in experiment is greater than it is in the control
x = sum(notnull_experiment.Payments / notnull_experiment.Clicks > notnull_control.Payments / notnull_control.Clicks)
n = len(notnull_experiment)

binom_test(x=x, n=n, p=0.5, cumulative=True, alternative='two-sided', formula=False)

0.6776

Sign Test

A: 
- The p-value of Gross conversion is 0.0026, which is less than 0.05. Therefore, Gross conversion is statistical significance.
- The p-value of Net conversion is 0.6776, which is greater than 0.05. Therefore, Net conversion is not statistical significance.